In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import sqlite3
import sqlalchemy
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker

In [2]:
years = list(range(2001, 2022))
url_data ="https://admin.euroleague.net/main/statistics?mode=Leaders&entity=Players&seasonmode=Single&seasoncode=E{}&cat=FieldGoalsMade2&agg=PerGame"

In [3]:
for year in years:
    with open("maded_two_pointers_htmls/{}.html".format(year), "w+" , encoding="utf-8") as file :
        url = url_data.format(year)
        data = requests.get(url)
        file.write(data.text)

In [5]:
dfs = []
for year in years:
    with open("maded_two_pointers_htmls/{}.html".format(year)) as f:
        page = f.read()
    
    soup = BeautifulSoup(page, 'html.parser')
    soup.find('tr', class_="StatsGridResults table responsive fixed-cols-2 table-left-cols-2 table-expand table-striped table-hover table-noborder table-centered table-condensed")
    player_table = soup.find_all(id="ctl00_ctl00_ctl00_ctl00_maincontainer_maincontent_contentpane_ctl01_ctl01_gvResults")[0]
    two_pointers_data = pd.read_html(str(player_table))[0]
    two_pointers_data["Year"] = year
    dfs.append(two_pointers_data)

In [6]:
two_pointers_data = pd.concat(dfs)
two_pointers_data

,Rank,Player,Team,GamesG,AttemptedAtt.,Made,% Made%,Year
0,1,"FORD, ALPHONSOFORD, A.",OLYOlympiacosOlympiacos,20,12.80,6.70,52.34%,2001
1,2,"GIRICEK, GORDANGIRICEK, G.",CSKCSKA MoscowCSKA Moscow,18,11.06,6.56,59.30%,2001
2,3,"KAMBALA, KASPARSKAMBALA, K.",EFSEfes PilsenEfes Pilsen,20,11.50,6.45,56.09%,2001
3,4,"STANOJEVIC, JOVOSTANOJEVIC, J.",PARPartizanPartizan,14,11.14,6.43,57.69%,2001
4,5,"BLAIR, JOSEPHBLAIR, J.",PESScavolini PesaroScavolini Pesaro,14,10.00,6.36,63.57%,2001
...,...,...,...,...,...,...,...,...
45,46,"MACON JR, DARYLMACON, D.",PAOPanathinaikos OPAP AthensPanathinaikos OPAP...,31,4.84,2.58,53.33%,2021
46,47,"DUNSTON, BRYANTDUNSTON, B.",EFSAnadolu Efes IstanbulAnadolu Efes Istanbul,37,3.57,2.57,71.97%,2021
47,48,"NUNNALLY, JAMESNUNNALLY, J.",MTAMaccabi Playtika Tel AvivMaccabi Playtika T...,32,5.03,2.53,50.31%,2021
48,49,"THOMAS, WILLTHOMAS, W.",ASMAS MonacoAS Monaco,35,3.91,2.51,64.23%,2021


In [7]:
engine = create_engine('sqlite:///maded_two_pointers_leaders.db')
Session = sessionmaker(bind=engine)
session = Session()

In [9]:
two_pointers_data.to_sql(
    name = "maded_two_pointers_leaders",
    con = engine,
    index = True,
    if_exists = "replace"
)

1050

In [10]:
maded_two_pointers_leaders_select = session.execute('select * from "maded_two_pointers_leaders"')
maded_two_pointers_leaders_select.all()

[(0, 1, 'FORD, ALPHONSOFORD, A.', 'OLYOlympiacosOlympiacos', 20, 12.8, 6.7, '52.34%', 2001),
 (1, 2, 'GIRICEK, GORDANGIRICEK, G.', 'CSKCSKA MoscowCSKA Moscow', 18, 11.06, 6.56, '59.30%', 2001),
 (2, 3, 'KAMBALA, KASPARSKAMBALA, K.', 'EFSEfes PilsenEfes Pilsen', 20, 11.5, 6.45, '56.09%', 2001),
 (3, 4, 'STANOJEVIC, JOVOSTANOJEVIC, J.', 'PARPartizanPartizan', 14, 11.14, 6.43, '57.69%', 2001),
 (4, 5, 'BLAIR, JOSEPHBLAIR, J.', 'PESScavolini PesaroScavolini Pesaro', 14, 10.0, 6.36, '63.57%', 2001),
 (5, 6, 'SCOLA, LUISSCOLA, L.', 'BKNTau CeramicaTau Ceramica', 20, 9.0, 6.25, '69.44%', 2001),
 (6, 7, 'BODIROGA, DEJANBODIROGA, D.', 'PAOPanathinaikos AthensPanathinaikos Athens', 22, 9.55, 6.0, '62.86%', 2001),
 (7, 8, 'FUCKA, GREGORFUCKA, G.', 'FORSkipper BolognaSkipper Bologna', 20, 11.8, 5.95, '50.42%', 2001),
 (8, 9, 'GOREE, MARCUSGOREE, M.', 'FRAOpel SkylinersOpel Skyliners', 14, 11.29, 5.71, '50.63%', 2001),
 (9, 10, 'SKELIN, MATESKELIN, M.', 'MESKRKA Novo MestoKRKA Novo Mesto', 14, 9.21